In [7]:
from nipype.interfaces.io import BIDSDataGrabber
from nipype.interfaces.utility import IdentityInterface
import os
from bids import BIDSLayout
import nipype.interfaces.freesurfer as fs
from nipype import Node, Function, MapNode
import nipype.interfaces.fsl as fsl
from nipype.interfaces.io import DataSink
from niworkflows.interfaces.bids import BIDSDataGrabber
from niworkflows.utils.bids import collect_data, relative_to_root
from nipype.interfaces.io import SelectFiles
from nipype.pipeline import Workflow
from petprep_hmc.bids import collect_data

In [2]:
bids_dir = '/Users/martinnorgaard/Documents/dev_dir/PDE4B_BIDS/'
skip_bids_validator = True
participant_label = None
mc_fwhm = 10
no_resample = True
mc_thresh = 20
mc_start_time = 120

In [3]:
if os.path.exists(bids_dir):
        if not skip_bids_validator:
            layout = BIDSLayout(bids_dir, validate=True)
        else:
            layout = BIDSLayout(bids_dir, validate=False)
else:
        raise Exception('BIDS directory does not exist')

In [5]:
# HELPER FUNCTIONS
def update_list_frames(in_file, min_frame):   
    """  
    Function to update the list of frames to be used in the hmc workflow.

    Parameters
    ----------
    in_file : list of frames
    min_frame : minimum frame to use for the analysis (first frame after 2 min)

    Returns
    -------
    new_list : list of updated frames to be used in the hmc workflow

    """
    
    new_list = [in_file[min_frame]] * min_frame + in_file[min_frame:]
    return new_list

def update_list_transforms(in_file, min_frame):   
    """
    Function to update the list of transforms to be used in the hmc workflow.
    
    Parameters
    ----------
    in_file : list of transforms
    min_frame : minimum frame to use for the analysis (first frame after 2 min)

    Returns
    -------
    lta_list : list of updated transforms to be used in the hmc workflow
    """
    
    new_list = [in_file[min_frame]] * min_frame + in_file[min_frame:]
    lta_list = [ext.replace('nii.gz','lta') for ext in new_list]  
    return lta_list

def add_mc_ext(in_file):    
    """ 
    Function to add the mc extension to the list of file names.

    Parameters
    ----------
    in_file : file name to be updated

    Returns
    -------
    mc_list : list of updated file names with mc extension
    """
    
    if len(in_file) > 1:
        mc_list = [ext.replace('.nii.gz','_mc.nii') for ext in in_file] # list comphrehension
    else:
        mc_list = in_file.replace('.nii.gz','_mc.nii')
    return mc_list

def lta2mat(in_file):  
    """
    Function to convert the lta file to the fsl format (.mat).
    
    Parameters
    ----------
    in_file : list of lta files to be converted

    Returns
    -------
    mat_list : list of mat files
    """
    
    mat_list = [ext.replace('.lta','.mat') for ext in in_file]
    return mat_list 

def get_min_frame(json_file, mc_start_time):  
    """
    Function to extract the frame number after mc_start_time (default=120) seconds of mid frames of dynamic PET data to be used with motion correction
        
    Parameters
    ----------
    json_file : json file containing the frame length and duration of the dynamic PET data

    Returns
    -------
    min_frame : minimum frame to use for the motion correction (first frame after 2 min)
    """  
    
    import os
    from os.path import join, isfile
    import numpy as np
    import json

    with open(json_file, 'r') as f:
        info = json.load(f)
        frames_duration = np.array(info['FrameDuration'], dtype=float)
        frames_start =np.pad(np.cumsum(frames_duration)[:-1],(1,0))
        mid_frames = frames_start + frames_duration/2

        min_frame = next(x for x, val in enumerate(mid_frames)
                                  if val > mc_start_time)    
    return min_frame  


def combine_hmc_outputs(translations, rot_angles, rotation_translation_matrix, in_file):   
    """
    
    Function to combine the outputs of the hmc workflow.

    Parameters
    ----------
    translations : list of estimated translations across frames
    rot_angles : list of estimated rotational angles across frames
    rotation_translation_matrix : list of estimated rotation translation matrices across frames
    in_file : list of frames to be used in the hmc workflow

    Returns
    -------
    Output path to confounds file for head motion correction
    """
    
    import os
    import pandas as pd
    import numpy as np
    import nibabel as nib
    
    new_pth = os.getcwd()
    
    movement = []
    for idx, trans in enumerate(translations):
        
        img = nib.load(in_file[idx])
        vox_ind = np.asarray(np.nonzero(img.get_fdata()))
        pos_bef = np.concatenate((vox_ind,np.ones((1,len(vox_ind[0,:])))))
        pos_aft = rotation_translation_matrix[idx] @ pos_bef
        diff_pos = pos_bef-pos_aft
        
        max_x = abs(max(diff_pos[0,:], key=abs))
        max_y = abs(max(diff_pos[1,:], key=abs))
        max_z = abs(max(diff_pos[2,:], key=abs))
        overall = np.sqrt(diff_pos[0,:] ** 2 + diff_pos[1,:] ** 2 + diff_pos[2,:] ** 2)
        max_tot = np.max(overall)
        median_tot = np.median(overall)
        
        movement.append(np.concatenate((translations[idx],rot_angles[idx], [max_x, max_y, max_z, max_tot, median_tot])))
        
    confounds = pd.DataFrame(movement, columns=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'max_x', 'max_y', 
                                                'max_z', 'max_tot', 'median_tot'])
    confounds.to_csv(os.path.join(new_pth,'hmc_confounds.tsv'), sep='\t')
    # np.savetxt(os.path.join(new_pth,'hmc_confounds.tsv'), movement, fmt='%10.5f', delimiter='\t', header='trans_x    trans_y    trans_z    rot_x    rot_y    rot_z')
    
    return os.path.join(new_pth,'hmc_confounds.tsv')

def plot_motion_outputs(in_file):   
    """
    Function to plot estimated motion data
    
    Parameters 
    ----------
    in_file : list of estimated motion data

    Returns
    -------
    Plots of estimated motion data
    """
    
    import os
    import pandas as pd
    import numpy as np
    import nibabel as nib
    import matplotlib.pyplot as plt
    
    confounds = pd.read_csv(in_file, sep='\t')
    
    new_pth = os.getcwd()
    
    n_frames = len(confounds.index)
    
    plt.figure(figsize=(11,5))
    plt.plot(np.arange(0,n_frames), confounds['trans_x'], "-r", label='trans_x')
    plt.plot(np.arange(0,n_frames), confounds['trans_y'], "-g", label='trans_y')
    plt.plot(np.arange(0,n_frames), confounds['trans_z'], "-b", label='trans_z')
    plt.legend(loc="upper left")
    plt.ylabel('Translation [mm]')
    plt.xlabel('frame #')
    plt.grid(visible=True)
    plt.savefig(os.path.join(new_pth,'translation.png'), format='png')
    plt.close()
    
    plt.figure(figsize=(11,5))
    plt.plot(np.arange(0,n_frames), confounds['rot_x'], "-r", label='rot_x')
    plt.plot(np.arange(0,n_frames), confounds['rot_y'], "-g", label='rot_y')
    plt.plot(np.arange(0,n_frames), confounds['rot_z'], "-b", label='rot_z')
    plt.legend(loc="upper left")
    plt.ylabel('Rotation [degrees]')
    plt.xlabel('frame #')
    plt.grid(visible=True)
    plt.savefig(os.path.join(new_pth,'rotation.png'), format='png')
    plt.close()
    
    plt.figure(figsize=(11,5))
    plt.plot(np.arange(0,n_frames), confounds['max_x'], "--r", label='max_x')
    plt.plot(np.arange(0,n_frames), confounds['max_y'], "--g", label='max_y')
    plt.plot(np.arange(0,n_frames), confounds['max_z'], "--b", label='max_z')
    plt.plot(np.arange(0,n_frames), confounds['max_tot'], "-k", label='max_total')
    plt.plot(np.arange(0,n_frames), confounds['median_tot'], "-m", label='median_tot')
    plt.legend(loc="upper left")
    plt.ylabel('Movement [mm]')
    plt.xlabel('frame #')
    plt.grid(visible=True)
    plt.savefig(os.path.join(new_pth,'movement.png'), format='png')
    plt.close()

def check_fsl_installed():
    try:
        fsl_home = os.environ['FSLDIR']
        if fsl_home:
            print("FSL is installed at:", fsl_home)
            return True
    except KeyError:
        print("FSL is not installed or FSLDIR environment variable is not set.")
        return False
    
# This function will strip the extension(s) from a filename
def strip_extensions(filename):
    while os.path.splitext(filename)[1]:
        filename = os.path.splitext(filename)[0]
    return filename

In [ ]:
def init_petprep_hmc_wf():
    petprep_hmc_wf = Workflow(name='petprep_hmc_wf', base_dir=bids_dir)

    # Define the subjects to iterate over
    subject_list = layout.get(return_type='id', target='subject', suffix='pet')

    # Set up the main workflow to iterate over subjects
    for subject_id in subject_list:
        # For each subject, create a subject-specific workflow
        subject_wf = init_single_subject_wf(subject_id)
        petprep_hmc_wf.add_nodes([subject_wf])

    return petprep_hmc_wf

def init_single_subject_wf(subject_id):
    # Create a new workflow for this specific subject
    subject_wf = Workflow(name=f'subject_{subject_id}_wf', base_dir='.')
    subject_wf.config['execution']['remove_unnecessary_outputs'] = 'false'
    
    subject_data = collect_data(layout,
                            participant_label=subject_id)[0]['pet']
    
    # This function will strip the extension(s) from a filename
    def strip_extensions(filename):
        while os.path.splitext(filename)[1]:
            filename = os.path.splitext(filename)[0]
        return filename

    # Use os.path.basename to get the last part of the path and then remove the extensions
    cleaned_subject_data = [strip_extensions(os.path.basename(path)) for path in subject_data]
    
    
    inputs = Node(IdentityInterface(fields=['pet_file']), name='inputs')
    inputs.iterables = ('pet_file', cleaned_subject_data)
    
    sessions = layout.get_sessions(subject=subject_id)
    
    
    templates = {'pet_file': 's*/pet/*{pet_file}.[n]*' if not sessions else 's*/s*/pet/*{pet_file}.[n]*',
                'json_file': 's*/pet/*{pet_file}.json' if not sessions else 's*/s*/pet/*{pet_file}.json'}
               
    selectfiles = Node(SelectFiles(templates, 
                               base_directory = bids_dir), 
                               name = "select_files")
                 
    split_pet = Node(interface = fs.MRIConvert(split = True), 
                     name = 'split_pet')
    
    smooth_frame = MapNode(interface=fsl.Smooth(fwhm=int(mc_fwhm)), 
                           name='smooth_frame', 
                           iterfield=['in_file'])
    
    thres_frame = MapNode(interface = fsl.maths.Threshold(thresh = int(mc_thresh), use_robust_range = True),
                          name = "thres_frame", 
                          iterfield = ['in_file'])
    
    estimate_motion = Node(interface = fs.RobustTemplate(auto_detect_sensitivity = True,
                                            intensity_scaling = True,
                                            average_metric = 'mean',
                                            args = '--cras'),
                           name="estimate_motion", iterfield=['in_files'])
    
    correct_motion = MapNode(interface = fs.ApplyVolTransform(), 
                             name = "correct_motion", 
                             iterfield = ['source_file', 'reg_file', 'transformed_file'])
    
    if no_resample:
        correct_motion.inputs.no_resample = True
    
    concat_frames = Node(interface = fs.Concatenate(concatenated_file = 'mc.nii.gz'), 
                         name = "concat_frames")
    
    lta2xform = MapNode(interface = fs.utils.LTAConvert(), 
                        name = "lta2xform", 
                        iterfield = ['in_lta', 'out_fsl'])
    
    est_trans_rot = MapNode(interface = fsl.AvScale(all_param = True), 
                            name = "est_trans_rot", 
                            iterfield = ['mat_file'])
    
    est_min_frame = Node(Function(input_names = ['json_file', 'mc_start_time'],
                                  output_names = ['min_frame'],
                                  function = get_min_frame),
                         name = "est_min_frame",)
    
    est_min_frame.inputs.mc_start_time = int(mc_start_time)

    upd_frame_list = Node(Function(input_names = ['in_file','min_frame'],
                                   output_names = ['upd_list_frames'],
                                   function = update_list_frames),
                          name = "upd_frame_list")
    
    upd_transform_list = Node(Function(input_names = ['in_file','min_frame'],
                                       output_names = ['upd_list_transforms'],
                                       function = update_list_transforms),
                          name = "upd_transform_list")
    
    hmc_movement_output = Node(Function(input_names = ['translations', 'rot_angles', 'rotation_translation_matrix','in_file'],
                                           output_names = ['hmc_confounds'],
                                           function = combine_hmc_outputs),
                               name = "hmc_movement_output")
    
    plot_motion = Node(Function(input_names = ['in_file'],
                                           function = plot_motion_outputs),
                               name = "plot_motion")

    # You might want to set up processing nodes for each PET file
    # For example, you could loop over PET files and connect nodes for each file here

    # Connect the BIDSDataGrabber to the rest of the subject workflow
    subject_wf.connect([(inputs, selectfiles, [('pet_file', 'pet_file')]),
                        (selectfiles, split_pet, [('pet_file', 'in_file')]),
                        (selectfiles, est_min_frame, [('json_file', 'json_file')]),
                        (split_pet,smooth_frame,[('out_file', 'in_file')]),
                        (smooth_frame,thres_frame,[('smoothed_file', 'in_file')]),
                        (thres_frame,upd_frame_list,[('out_file', 'in_file')]),
                        (est_min_frame,upd_frame_list,[('min_frame', 'min_frame')]),
                        (upd_frame_list,estimate_motion,[('upd_list_frames', 'in_files')]),
                        (thres_frame,upd_transform_list,[('out_file', 'in_file')]),
                        (est_min_frame,upd_transform_list,[('min_frame', 'min_frame')]),
                        (upd_transform_list,estimate_motion,[('upd_list_transforms', 'transform_outputs')]),
                        (split_pet,correct_motion,[('out_file', 'source_file')]),
                        (estimate_motion,correct_motion,[('transform_outputs', 'reg_file')]),
                        (estimate_motion,correct_motion,[('out_file', 'target_file')]),
                        (split_pet,correct_motion,[(('out_file', add_mc_ext), 'transformed_file')]),
                        (correct_motion,concat_frames,[('transformed_file', 'in_files')]),
                        (estimate_motion,lta2xform,[('transform_outputs', 'in_lta')]),
                        (estimate_motion,lta2xform,[(('transform_outputs', lta2mat), 'out_fsl')]),
                        (lta2xform,est_trans_rot,[('out_fsl', 'mat_file')]),
                        (est_trans_rot,hmc_movement_output,[('translations', 'translations'),('rot_angles', 'rot_angles'),('rotation_translation_matrix','rotation_translation_matrix')]),
                        (upd_frame_list,hmc_movement_output,[('upd_list_frames', 'in_file')]),
                        (hmc_movement_output,plot_motion,[('hmc_confounds','in_file')]),
    ])

    return subject_wf

# Run the main workflow
main = init_petprep_hmc_wf()
main.config['execution']['remove_unnecessary_outputs'] = 'false'
main.run(plugin="MultiProc", plugin_args={"n_proc": 8})




231108-14:06:39,144 nipype.workflow INFO:
	 Workflow petprep_hmc_wf settings: ['check', 'execution', 'logging', 'monitoring']
231108-14:06:39,277 nipype.workflow INFO:
	 Running in parallel.
231108-14:06:39,471 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 18 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 10/10.
231108-14:06:41,476 nipype.workflow INFO:
	 [MultiProc] Running 10 tasks, and 8 jobs ready. Free memory (GB): 26.80/28.80, Free processors: 0/10.
                     Currently running:
                       * petprep_hmc_wf.subject_PF08_wf.select_files
                       * petprep_hmc_wf.subject_PF08_wf.select_files
                       * petprep_hmc_wf.subject_PF09_wf.select_files
                       * petprep_hmc_wf.subject_PF09_wf.select_files
                       * petprep_hmc_wf.subject_PF10_wf.select_files
                       * petprep_hmc_wf.subject_PF10_wf.select_files
                       * petprep_hmc_wf.subject_PF11_wf.select

231108-09:22:27,609 nipype.workflow INFO:
	 [Node] Finished "_thres_frame3", elapsed time 0.405997s.
231108-09:22:27,639 nipype.workflow INFO:
	 [Node] Finished "_thres_frame4", elapsed time 0.427634s.
231108-09:22:27,644 nipype.workflow INFO:
	 [Node] Finished "_thres_frame1", elapsed time 0.441799s.
231108-09:22:27,667 nipype.workflow INFO:
	 [Node] Finished "_thres_frame7", elapsed time 0.455737s.
231108-09:22:27,679 nipype.workflow INFO:
	 [Node] Finished "_thres_frame8", elapsed time 0.470162s.
231108-09:22:27,680 nipype.workflow INFO:
	 [Node] Finished "_thres_frame6", elapsed time 0.469893s.
231108-09:22:27,688 nipype.workflow INFO:
	 [Node] Finished "_thres_frame2", elapsed time 0.486889s.
231108-09:22:27,702 nipype.workflow INFO:
	 [Node] Finished "_thres_frame5", elapsed time 0.496665s.
231108-09:22:27,718 nipype.workflow INFO:
	 [Node] Finished "_thres_frame0", elapsed time 0.519845s.
231108-09:22:28,70 nipype.workflow INFO:
	 [Node] Finished "_smooth_frame20", elapsed time 

231108-09:22:31,142 nipype.workflow INFO:
	 [Job 15] Completed (petprep_hmc_wf.subject_01_wf.smooth_frame).
231108-09:22:31,143 nipype.workflow INFO:
	 [Job 149] Completed (_thres_frame9).
231108-09:22:31,144 nipype.workflow INFO:
	 [Job 150] Completed (_thres_frame10).
231108-09:22:31,144 nipype.workflow INFO:
	 [Job 151] Completed (_thres_frame11).
231108-09:22:31,144 nipype.workflow INFO:
	 [Job 152] Completed (_thres_frame12).
231108-09:22:31,145 nipype.workflow INFO:
	 [Job 153] Completed (_thres_frame13).
231108-09:22:31,145 nipype.workflow INFO:
	 [Job 154] Completed (_thres_frame14).
231108-09:22:31,146 nipype.workflow INFO:
	 [Job 155] Completed (_thres_frame15).
231108-09:22:31,146 nipype.workflow INFO:
	 [Job 156] Completed (_thres_frame16).
231108-09:22:31,147 nipype.workflow INFO:
	 [Job 157] Completed (_thres_frame17).
231108-09:22:31,148 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 46 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 10/10.
231108-0

231108-09:22:33,599 nipype.workflow INFO:
	 [Node] Finished "_thres_frame10", elapsed time 0.382376s.
231108-09:22:33,627 nipype.workflow INFO:
	 [Node] Finished "_thres_frame8", elapsed time 0.416247s.
231108-09:22:33,629 nipype.workflow INFO:
	 [Node] Finished "_thres_frame7", elapsed time 0.418178s.
231108-09:22:33,634 nipype.workflow INFO:
	 [Node] Finished "_thres_frame13", elapsed time 0.410263s.
231108-09:22:33,655 nipype.workflow INFO:
	 [Node] Finished "_thres_frame12", elapsed time 0.436038s.
231108-09:22:33,657 nipype.workflow INFO:
	 [Node] Finished "_thres_frame9", elapsed time 0.443573s.
231108-09:22:33,674 nipype.workflow INFO:
	 [Node] Finished "_thres_frame15", elapsed time 0.448634s.
231108-09:22:33,679 nipype.workflow INFO:
	 [Node] Finished "_thres_frame14", elapsed time 0.456063s.
231108-09:22:33,679 nipype.workflow INFO:
	 [Node] Finished "_thres_frame11", elapsed time 0.460528s.
231108-09:22:35,147 nipype.workflow INFO:
	 [Job 16] Completed (petprep_hmc_wf.subjec

231108-09:22:37,634 nipype.workflow INFO:
	 [Node] Finished "_thres_frame10", elapsed time 0.401337s.
231108-09:22:37,644 nipype.workflow INFO:
	 [Node] Finished "_thres_frame6", elapsed time 0.420425s.
231108-09:22:37,644 nipype.workflow INFO:
	 [Node] Finished "_thres_frame3", elapsed time 0.427005s.
231108-09:22:37,649 nipype.workflow INFO:
	 [Node] Finished "_thres_frame4", elapsed time 0.430139s.
231108-09:22:37,658 nipype.workflow INFO:
	 [Node] Finished "_thres_frame7", elapsed time 0.433223s.
231108-09:22:37,662 nipype.workflow INFO:
	 [Node] Finished "_thres_frame8", elapsed time 0.438226s.
231108-09:22:37,668 nipype.workflow INFO:
	 [Node] Finished "_thres_frame5", elapsed time 0.450157s.
231108-09:22:37,677 nipype.workflow INFO:
	 [Node] Finished "_thres_frame9", elapsed time 0.453654s.
231108-09:22:39,156 nipype.workflow INFO:
	 [Job 17] Completed (petprep_hmc_wf.subject_01_wf.thres_frame).
231108-09:22:39,157 nipype.workflow INFO:
	 [Job 185] Completed (_thres_frame3).
231

...

	 [Node] Finished "_correct_motion14", elapsed time 0.131444s.
231108-09:30:02,637 nipype.workflow INFO:
	 [Node] Setting-up "_correct_motion15" in "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/petprep_hmc/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/correct_motion/mapflow/_correct_motion15".
231108-09:30:02,638 nipype.workflow INFO:
	 [Node] Outdated cache found for "_correct_motion15".
231108-09:30:02,642 nipype.workflow INFO:
	 [Node] Executing "_correct_motion15" <nipype.interfaces.freesurfer.preprocess.ApplyVolTransform>
231108-09:30:02,777 nipype.workflow INFO:
	 [Node] Finished "_correct_motion15", elapsed time 0.133908s.
231108-09:30:02,779 nipype.workflow INFO:
	 [Node] Setting-up "_correct_motion16" in "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/petprep_hmc/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/correct_motion/mapflow/_correct_motion16".
231108-09:30:02,779 nipype.workflow INFO:
	 [Node] Outdated cache found fo

231108-09:30:05,97 nipype.workflow INFO:
	 [Node] Finished "_lta2xform7", elapsed time 0.096538s.
231108-09:30:05,99 nipype.workflow INFO:
	 [Node] Setting-up "_lta2xform8" in "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/petprep_hmc/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/lta2xform/mapflow/_lta2xform8".
231108-09:30:05,100 nipype.workflow INFO:
	 [Node] Outdated cache found for "_lta2xform8".
231108-09:30:05,102 nipype.workflow INFO:
	 [Node] Executing "_lta2xform8" <nipype.interfaces.freesurfer.utils.LTAConvert>
231108-09:30:05,205 nipype.workflow INFO:
	 [Node] Finished "_lta2xform8", elapsed time 0.102885s.
231108-09:30:05,208 nipype.workflow INFO:
	 [Node] Setting-up "_lta2xform9" in "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/petprep_hmc/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/lta2xform/mapflow/_lta2xform9".
231108-09:30:05,209 nipype.workflow INFO:
	 [Node] Outdated cache found for "_lta2xform9".
231108-09:30:

231108-09:30:10,228 nipype.workflow INFO:
	 [Node] Setting-up "_est_trans_rot0" in "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/petprep_hmc/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/est_trans_rot/mapflow/_est_trans_rot0".
231108-09:30:10,228 nipype.workflow INFO:
	 [Node] Setting-up "_est_trans_rot1" in "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/petprep_hmc/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/est_trans_rot/mapflow/_est_trans_rot1".
231108-09:30:10,229 nipype.workflow INFO:
	 [Node] Setting-up "_est_trans_rot2" in "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/petprep_hmc/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/est_trans_rot/mapflow/_est_trans_rot2".
231108-09:30:10,230 nipype.workflow INFO:
	 [Node] Executing "_est_trans_rot1" <nipype.interfaces.fsl.utils.AvScale>
231108-09:30:10,230 nipype.workflow INFO:
	 [Node] Executing "_est_trans_rot0" <nipype.interfaces.fsl.utils.AvScale>
2

231108-09:30:12,397 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot13", elapsed time 0.1532s.
231108-09:30:12,445 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot14", elapsed time 0.199719s.
231108-09:30:12,457 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot16", elapsed time 0.205342s.
231108-09:30:12,465 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot10", elapsed time 0.224596s.
231108-09:30:12,466 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot15", elapsed time 0.219636s.
231108-09:30:12,468 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot11", elapsed time 0.228728s.
231108-09:30:12,483 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot19", elapsed time 0.227203s.
231108-09:30:12,483 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot17", elapsed time 0.236517s.
231108-09:30:12,490 nipype.workflow INFO:
	 [Node] Finished "_est_trans_rot18", elapsed time 0.243739s.
231108-09:30:12,492 nipype.workflow INFO:
	 [Node] Finished "_est_

231108-09:30:18,184 nipype.workflow INFO:
	 [Job 43] Completed (petprep_hmc_wf.subject_01_wf.est_trans_rot).
231108-09:30:18,186 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 10/10.
231108-09:30:18,240 nipype.workflow INFO:
	 [Node] Setting-up "petprep_hmc_wf.subject_01_wf.hmc_movement_output" in "/Users/martinnorgaard/Dropbox/Mac/Documents/GitHub/petprep_hmc/data/petprep_hmc_wf/subject_01_wf/_pet_file_sub-01_ses-baseline_pet/hmc_movement_output".
231108-09:30:18,251 nipype.workflow INFO:
	 [Node] Executing "hmc_movement_output" <nipype.interfaces.utility.wrappers.Function>
231108-09:30:20,189 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 9/10.
                     Currently running:
                       * petprep_hmc_wf.subject_01_wf.hmc_movement_output
231108-09:30:21,212 nipype.workflow INFO:
	 [Node] Finished "hmc_movement_output", elap

In [50]:
from nilearn.plotting import plot_epi
import nibabel as nib
from nilearn import image
from nilearn.plotting.find_cuts import find_xyz_cut_coords
import numpy as np
from niworkflows.interfaces.bids import BIDSInfo
import imageio
import os

def plot_mc_dynamic_pet(pet_orig, pet_mc, sub_out_dir, file_prefix):
    nib_orig = nib.load(pet_orig)
    nib_mc = nib.load(pet_mc)
    shape_orig = nib_orig.shape
    shape_mc = nib_mc.shape
    t_orig = shape_orig[-1]
    t_mc = shape_mc[-1]

    mid_orig = image.index_img(pet_orig, t_orig//2)
    mid_mc = image.index_img(pet_mc, t_mc//2)

    vmax_orig = np.percentile(mid_orig.get_fdata().flatten(), 99.9) 
    vmin_orig = np.percentile(mid_orig.get_fdata().flatten(), 80)
    vmax_mc = np.percentile(mid_mc.get_fdata().flatten(), 99.9)
    vmin_mc = np.percentile(mid_mc.get_fdata().flatten(), 80)

    x_orig, y_orig, z_orig = find_xyz_cut_coords(mid_orig)
    x_mc, y_mc, z_mc = find_xyz_cut_coords(mid_mc)

    orig_images = []
    for idx, img in enumerate(image.iter_img(pet_orig)):
        # img is now an in-memory 3D img
        plot_epi(
            img, colorbar=True, display_mode='ortho', title=f"Frame #{idx}", cut_coords=(x_orig,y_orig,z_orig), vmin=vmin_orig, vmax=vmax_orig, output_file=f"orig_{idx}.png"
        )
        orig_images.append(imageio.imread(f"orig_{idx}.png"))
        os.remove(f"orig_{idx}.png")
    
    # Write the images to a GIF file
    imageio.mimsave(os.path.join(sub_out_dir, f'{file_prefix}_desc-without_motion_correction.gif'), orig_images, duration=1, loop=0)

    mc_images = []
    for idx, img in enumerate(image.iter_img(pet_mc)):
        # img is now an in-memory 3D img
        plot_epi(
            img, colorbar=True, display_mode='ortho', title=f"Frame #{idx}", cut_coords=(x_mc,y_mc,z_mc), vmin=vmin_mc, vmax=vmax_mc, output_file=f"mc_{idx}.png"
        )
        mc_images.append(imageio.imread(f"mc_{idx}.png"))
        os.remove(f"mc_{idx}.png")
    
    # Write the images to a GIF file
    imageio.mimsave(os.path.join(sub_out_dir, f'{file_prefix}_desc-with_motion_correction.gif'), mc_images, duration=1, loop=0)

    

In [52]:
import glob
from pathlib import Path
import re
import shutil
import sys
import json

output_dir = os.path.join(bids_dir,'derivatives','petprep_hmc')

# loop through directories and store according to BIDS
mc_files = glob.glob(os.path.join(Path(bids_dir),'petprep_hmc_wf','*','*','*','mc.nii.gz'))
confound_files = glob.glob(os.path.join(Path(bids_dir),'petprep_hmc_wf','*','*','*','hmc_confounds.tsv'))
movement = glob.glob(os.path.join(Path(bids_dir),'petprep_hmc_wf','*','*','*','movement.png'))
rotation = glob.glob(os.path.join(Path(bids_dir),'petprep_hmc_wf','*','*','*','rotation.png'))
translation = glob.glob(os.path.join(Path(bids_dir),'petprep_hmc_wf','*','*','*','translation.png'))

for idx, x in enumerate(mc_files):
    match_sub_id = re.search(r'sub-([A-Za-z0-9]+)_', mc_files[idx])
    sub_id = match_sub_id.group(1)
    
    match_ses_id = re.search(r'ses-([A-Za-z0-9]+)_', mc_files[idx])
    ses_id = match_ses_id.group(1)
    
    match_file_prefix = re.search(r'_pet_file_(.*?)_pet', mc_files[idx])
    file_prefix = match_file_prefix.group(1)
    
    if ses_id:
        sub_out_dir = Path(os.path.join(output_dir, 'sub-' + sub_id, 'ses-' + ses_id))
    else:
        sub_out_dir = Path(os.path.join(output_dir, 'sub-' + sub_id))
    
    os.makedirs(sub_out_dir, exist_ok=True)
    shutil.copyfile(mc_files[idx], os.path.join(sub_out_dir, f'{file_prefix}_desc-mc_pet.nii.gz'))
    shutil.copyfile(confound_files[idx], os.path.join(sub_out_dir, f'{file_prefix}_desc-confounds_timeseries.tsv'))
    shutil.copyfile(movement[idx], os.path.join(sub_out_dir, f'{file_prefix}_movement.png'))
    shutil.copyfile(rotation[idx], os.path.join(sub_out_dir, f'{file_prefix}_rotation.png'))
    shutil.copyfile(translation[idx], os.path.join(sub_out_dir, f'{file_prefix}_translation.png'))
        
    if ses_id:
        source_file = layout.get(suffix='pet', subject=sub_id, session=ses_id, extension=['.nii', '.nii.gz'], return_type='filename')[0]
    else:
        source_file = layout.get(suffix='pet', subject=sub_id, extension=['.nii', '.nii.gz'], return_type='filename')[0]

    hmc_json = {
        "Description": "Motion-corrected PET file",
        "Sources": source_file,
        "ReferenceImage": "Robust template using mri_robust_register",
        "CostFunction": "ROB",
        "MCTreshold": f"{mc_thresh}",
        "MCFWHM": f"{mc_fwhm}",
        "MCStartTime": f"{mc_start_time}",
        "QC": "",
        "SoftwareName": "PETPrep HMC workflow",
        "SoftwareVersion": 'test',
        "CommandLine": " ".join(sys.argv)
        }
        
    json_object = json.dumps(hmc_json, indent=4)
    with open(os.path.join(sub_out_dir, f'{file_prefix}_desc-mc_pet.json'), "w") as outfile:
        outfile.write(json_object)

    # Plot with and without motion correction
    plot_mc_dynamic_pet(source_file, mc_files[idx], sub_out_dir, file_prefix)
        
# remove temp outputs
shutil.rmtree(os.path.join(bids_dir, 'petprep_hmc_wf'))

/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  orig_images.append(imageio.imread(f"orig_{idx}.png"))
/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mc_images.append(imageio.imread(f"mc_{idx}.png"))
/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (an

/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  orig_images.append(imageio.imread(f"orig_{idx}.png"))
/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mc_images.append(imageio.imread(f"mc_{idx}.png"))
/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (an

/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  orig_images.append(imageio.imread(f"orig_{idx}.png"))
/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mc_images.append(imageio.imread(f"mc_{idx}.png"))
/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (an

/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  orig_images.append(imageio.imread(f"orig_{idx}.png"))
/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mc_images.append(imageio.imread(f"mc_{idx}.png"))
/var/folders/0d/tx6_gy7951749z57rjjkkl_m0000gn/T/ipykernel_86237/3675448933.py:47: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (an